In [99]:
import torch.nn as nn
import torch
from torch.optim import RMSprop
import math
from dataclasses import dataclass
from typing import Optional, Tuple, List, Dict
import numpy as np
from scipy.optimize import minimize
from tqdm import tqdm
import pandas as pd
import os
from datetime import datetime
import functions
from functions import *
import importlib

importlib.reload(functions)

<module 'functions' from 'c:\\Users\\ADMIN\\Desktop\\FPTS\\Modeling Time Series Risks\\scripts\\functions.py'>

In [7]:
# ticker_list = ['REE', 'SAM', 'HAP', 'GMD', 'GIL', 'TMS', 'SAV', 'DHA', 'MHC', 'HAS'] # 10 stocks with the most observations
ticker_list = ['REE', 'SAM', 'HAP'] # 3 stocks with the most observations
limits = {
    'hose':0.07,
    'hnx':0.1,
    'upcom':0.15
}
# Read and merge into 1 dataset

if "stock_data.csv" in os.listdir("data"):
    merged_df = pd.read_csv(
        os.path.join("data", "stock_data.csv"),
        index_col=None
    ).assign(
        date = lambda df : pd.to_datetime(df["date"])
    )
else:
    # Read and merge data
    hnx = pd.read_csv(os.path.join("data", "CafeF.HNX.Upto31.07.2025.csv")).assign(
        floor = "hnx"
    )
    hsx = pd.read_csv(os.path.join("data", "CafeF.HSX.Upto31.07.2025.csv")).assign(
        floor = "hose"
    )
    upcom = pd.read_csv(os.path.join("data", "CafeF.UPCOM.Upto31.07.2025.csv")).assign(
        floor = "upcom"
    )
    indexes = pd.read_csv(os.path.join("data", "CafeF.INDEX.Upto06.08.2025.csv")).assign(
        floor = "index"
    )

    # Rename columns
    hnx, hsx, upcom, indexes = [
        df.rename(columns={
            "<Ticker>":"ticker",
            "<DTYYYYMMDD>":"date",
            "<Open>":"open",
            "<High>":"high",
            "<Low>":"low",
            "<Close>":"close",
            "<Volume>":"volume"
        }) for df in [hnx, hsx, upcom, indexes]
    ]
        
    # Merge and clean data
    # UPCOM has missing tickers for some reason
    merged_df = pd.concat(
        [hnx, hsx, upcom, indexes],
        axis=0
    ).reset_index(drop=True).dropna(subset="ticker")\
    .assign(
        date=lambda df : df["date"].astype(str).apply(lambda x: datetime.strptime(x, "%Y%m%d").date())
    )
    merged_df.to_csv(
        os.path.join("data", "stock_data.csv"),
        index=False
    ) # Save merged data to save time in future runs


# Data cleaning and merging

data = merged_df[["date", "ticker", "floor", "close"]].sort_values(["ticker", "date"]).assign(
    returns = lambda df : df.groupby("ticker")["close"].pct_change(),
    log_returns_pct = lambda df : np.log(df["close"] / df.groupby("ticker")["close"].shift(1))*100
)

data = data.loc[data["ticker"].str.len()==3] # Eliminate ETF, and indeces

data["limit"] = data["floor"].map(limits)
outliers = data.loc[data["returns"].abs() > data["limit"]]
clean_df = data.drop(outliers.index) # Remove outliers
print(f"% of observations removed: {round((len(outliers)/len(data))*100, 2)}%")

# NOTE: try out different samples of stocks
pivoted_df = clean_df.pivot_table(values="returns", index="date", columns="ticker") # Pivot data for better usability
pivoted_df = pivoted_df[ticker_list].dropna()

display(pivoted_df.describe())
train_df, test_df = split_train_test(pivoted_df)

% of observations removed: 1.05%


ticker,REE,SAM,HAP
count,5951.000000,5951.000000,5951.000000
mean,0.001091,0.000699,0.000770
std,0.021411,0.023733,0.024894
min,-0.069971,-0.069999,-0.069963
25%,-0.009689,-0.011620,-0.012434
50%,0.000000,0.000000,0.000000
75%,0.011761,0.012037,0.012855
max,0.069962,0.069919,0.069927


In [ ]:
def to_tensor(x, device):
    if isinstance(x, np.ndarray):
        return torch.from_numpy(x).to(device=device, dtype=torch.float64)
    if isinstance(x, torch.Tensor):
        return x.to(device=device, dtype=torch.float64)
    raise TypeError("Unsupported array type")

def lower_triangle_index(n_assets: int):
    '''
    Get index of lower triangular matrix
    '''
    rows, cols = torch.tril_indices(n_assets, n_assets)
    diag_mask = rows == cols
    off_mask = ~diag_mask
    return rows, cols, diag_mask, off_mask

def vec_to_lower_tri(vec: torch.Tensor, n_assets: int) -> torch.Tensor:
    """
    Map vector of length n_assets(n_assets+1)/2 to a lower-triangular matrix
    Args:
        vec: shape (..., n_assets(n_assets+1)/2)
    Output:
        shapeL (..., n_assets,n_assets)
    """

    vec_length = n_assets * (n_assets + 1) // 2
    assert vec.shape[-1] == vec_length, f"Expected vector length {vec_length} for number of assets = {n_assets}, got {vec.shape[-1]}"
    rows, cols, _, _ = lower_triangle_index(n_assets)
    out = vec.new_zeros(*vec.shape[:-1], n_assets, n_assets) # Initialize
    out[..., rows, cols] = vec

    return out

class Static_C(nn.Module):
    '''
    Parameterization of the static lower-triangular C with positive diagonal to ensure semipositive definitenes of C'C
    '''
    def __init__(self, n_assets: int):
        super().__init__()
        self.n_assets = n_assets
        self.off_idx = torch.tril_indices(n_assets, n_assets, offset=-1)
        num_off = self.off_idx.shape[1]
        
        self.off_params = nn.Parameter(
            torch.zeros(num_off, dtype=torch.float64)
        )
        self.diag_params = nn.Parameter(
            torch.zeros(n_assets, dtype=torch.float64)
        )
    
    def forward(self) -> torch.Tensor:
        C = self.off_params.new_zeros(self.n_assets, self.n_assets)
        # Off diagonals parameteres
        C[self.off_idx[0], self.off_idx[1]] = self.off_params
        # Diagonal parameteres
        C[range(self.n_assets), range(self.n_assets)] = torch.nn.functional.softplus(self.diag_params) + 1e-6

        return C

class LSTM_Dynamic_C(nn.Module):
    """
    Map past returns to dynamic element C_t, map LSTM output to lower triangular matrix C_t, and regularize the diagonal with a Swish function Swish(x) = x*sigmopoid(Beta*x), with learnable x.
    The Swish function helps stability without forcing diagonals strictly positive
    """

    def __init__(self, n_assets: int, hidden_size: Optional[int] = None, num_layers: int = 1, dropout:float = 0.1):
        super().__init__()
        self.n_assets = n_assets
        self.length = n_assets * (n_assets + 1) // 2
        self.hidden_size = hidden_size or max(8, n_assets) # At least equal to number of assets
        self.num_layers = num_layers

        # LSTM that takes in r_{t-1} and outputs a vector length = self.length at each step
        self.lstm = nn.LSTM(
            input_size=n_assets,
            hidden_size=self.hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            batch_first=True,
            dtype=torch.float64
        )

        self.out = nn.Linear(
            self.hidden_size, 
            self.length, 
            dtype=torch.float64
        )
        self.beta = nn.Parameter(torch.tensor(1.0, dtype=torch.float64)) # Swish parameters beta, is learnable

    def swish(self, x: torch.Tensor) -> torch.Tensor:
        # Swish(x) = x * sigmoid(beta * x)
        return x * torch.sigmoid(self.beta * x)

    def forward(self, returns: torch.Tensor) -> torch.Tensor:
        """
        Args:
            returns: TxM returns
        Outputs:
            C_t for t = 1,... T shape (T, M, M)
        Convention: at step t we feed r_{t-1}, for t=0 we won't form C_0
        """

        n_periods, n_assets = returns.shape
        assert n_assets == self.n_assets

        x = returns.unsqueeze(0) # Add new dimension at the beginning to (1, T, M) to be suitable with torch
        h, _ = self.lstm(x) # (1, T, H)
        z = self.out(h) # (1, T, L)
        z = z.squeeze(0) # (T, L)
        C_full = vec_to_lower_tri(z, self.n_assets)

        # Apply Swish to only diagonal elements
        rows, cols, diag_mask, off_mask = lower_triangle_index(self.n_assets)
        diag_vals = C_full[..., rows[diag_mask], cols[diag_mask]] # Get all diagonal values
        diag_vals = self.swish(diag_vals) # Apply Swish
        C_full[..., rows[diag_mask], cols[diag_mask]] = diag_vals # Assign Swish-applied values to the lower-triangular matrix C

        return C_full # C_t
    
class params(nn.Module):
    """
    Static scalars a, b with constraints: a, b >= 0; a + b < 1
    Use positive reparameterization via softplus -> normalize:
        u = softplus(u0), v = softplus(v0); s = u + v + 1; a = u/s, b = v/s
    Which makes a, b in (0,1) and a+b <1
    """

    def __init__(self):
        super().__init__()
        self.u0 = nn.Parameter(
            torch.tensor(0.2, dtype=torch.float64)
        )
        self.v0 = nn.Parameter(
            torch.tensor(0.7, dtype=torch.float64)
        )
    
    def forward(self) -> Tuple[torch.Tensor, torch.Tensor]:
        u = torch.nn.functional.softplus(self.u0)
        v = torch.nn.functional.softplus(self.v0)
        s = u + v + 1.0
        a = u/s
        b = v/s
        return a, b
    

@dataclass
class LSTM_BEKK_config:
    hidden_size: Optional[int] = None
    num_layers: int = 1
    dropout: float = 0.1
    lr: float = 0.001
    weight_decay: float = 0.0
    epochs: int = 500
    grad_clip: float = 10.0
    val_split: float = 0.1
    early_stopping_patience: int = 20
    device: str = "cpu"
    jitter: float = 1e-8 # for Cholesky stability
    seed: int = 1

class LSTM_BEKK(nn.Module):
    """
    H_t = C*C' + C_t*C_t' + a*r_{t-1}*r_{t-1}' + b*H_{t-1}
    with Gaussian log-likelihood
    """

    def __init__(self, n_assets: int, config: Optional[LSTM_BEKK_config]=None):
        super().__init__()
        self.n_assets = n_assets
        self.config = config or LSTM_BEKK_config()
        torch.manual_seed(self.config.seed) # Set seed

        self.C = Static_C(n_assets)
        self.C_dynamic = LSTM_Dynamic_C(
            n_assets=n_assets, 
            hidden_size=self.config.hidden_size,
            num_layers=self.config.num_layers,
            dropout=self.config.dropout
        )
        self.ab = params()

        # Buffers created at fit-time 
        # NOTE: wtf is this ==================================================================
        self.H0_: Optional[torch.Tensor] = None
        self.mu_: Optional[torch.Tensor] = None
        
        self.to(
            dtype=torch.float64,
            device=self.config.device
        )
    
    def forward_sequence(
            self,
            returns: torch.Tensor,
            init_cov_matrix: Optional[torch.Tensor]=None
        ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Implement the recursion for covariance matrix H_t. Compute covariance matrix H_t and per-step negative log-likelihood terms, given returns TxM
        Output:
            H: (T, M, M)
            nll_terms: (T,) with 0.5 * (logdet(cov_matrix) + r_t' * cov_matrix **(-1)*r_t)
        """
        config = self.config
        device = config.device
        n_periods, n_assets = returns.shape
        assert n_assets == self.n_assets

        # Static C and static scalar a, b
        C = self.C()
        a, b = self.ab()

        # Dynamic C_t
        C_t_all = self.C_dynamic(returns)

        # Initialize H_0
        if init_cov_matrix is None:
            cov = torch.cov(returns.T)
            cov = cov + config.jitter * torch.eye(
                n_assets,
                dtype=torch.float64,
                device=device
            )
            prev_cov_matrix = cov
        else:
            prev_cov_matrix = init_cov_matrix.to(device=device, dtype=torch.float64)
        
        cov_matrix_list = []
        nll_terms = []

        eye_assets = torch.eye(n_assets, dtype=torch.float64, device=device)

        for t in range(n_periods):
            prev_returns = returns[t-1].unsqueeze(0).T if t>0 else torch.zeros(n_assets, 1, dtype=torch.float64, device=device)
            C_t = C_t_all[t]

            C_static = C @ C.T
            C_dynamic = C_t @ C_t.T
            arch = a * (prev_returns @ prev_returns.T)
            garch = b * prev_cov_matrix

            cov_matrix = C_static + C_dynamic + arch + garch

            # Stabilize for Cholesky
            # NOTE wtf is this? -==========================
            cov_matrix = cov_matrix + config.jitter * eye_assets

            # Cholesky-based log-likelihood
            L = torch.linalg.cholesky(cov_matrix)
            logdet = 2.0 * torch.log(torch.diag(L)).sum()

            # Solve H^{-1}*r via 2 triangular solves
            returns_t = returns[t].unsqueeze(0).T
            y = torch.cholesky_solve(returns_t, L) # Solves H*y=r
            quad = (returns_t.T @ y).squeeze() # r' * H^{t-1} * r

            nll_t = 0.5 * (logdet + quad)
            nll_terms.append(nll_t)
            cov_matrix_list.append(cov_matrix)

            prev_cov_matrix = cov_matrix
        
        cov_matrices = torch.stack(cov_matrix_list, dim=0)
        nll_terms = torch.stack(nll_terms, dim=0)

        return cov_matrices, nll_terms
    
    def negative_loglik(self, returns: torch.Tensor) -> torch.Tensor:

        _, nll_terms = self.forward_sequence(returns, init_cov_matrix=self.H0_)
        return nll_terms.sum()
    
    def fit(
            self,
            returns_df: pd.DataFrame,
            verbose: bool = True
    ) -> Dict[str, float]:
        """
        Fit the model by minimizing the Gaussian negative log-likelihood
        returns_df: TxM of demeaned returns
        """

        config = self.config
        device = config.device

        returns_np = returns_df.to_numpy(dtype=float)
        returns_tensor = to_tensor(returns_np, device=device)
        n_periods = returns_tensor.shape[0]

        # split train, valuate set
        t_valuate = max(1, int(math.floor(config.val_split * n_periods)))
        t_train = n_periods - t_valuate
        r_train = returns_tensor[:t_train]
        r_valuate = returns_tensor[t_train:]

        # set H_0 from training sample
        cov = torch.cov(r_train.T)
        cov = cov + config.jitter * torch.eye(
            self.n_assets,
            dtype=torch.float64,
            device=device
        )
        self.init_cov_matrix = cov.detach()

        params = list(self.parameters())
        opt = RMSprop(params, lr=config.lr, weight_decay=config.weight_decay)
        best_val = float("inf")
        best_state = None
        patience = config.early_stopping_patience
        epochs_no_improve = 0

        for epoch in range(config.epochs):
            self.train()
            opt.zero_grad(set_to_none=True)
            nll = self.negative_loglik(r_train)
            nll.backward()
            if config.grad_clip is not None and config.grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(self.parameters(), config.grad_clip)
            opt.step()

            # Evaluate on validation
            self.eval()
            with torch.no_grad():
                val_nll = self.negative_loglik(r_valuate).item()

            if verbose and (epoch % 10 == 0 or epoch == config.epochs-1):
                print(f"[{epoch:04d}] train NLL : {nll.item():.3f} | val NLL : {val_nll:.3f}")

            # Early stopping
            if val_nll + 1e-9 < best_val:
                best_val = val_nll
                best_state = {
                    k: v.detach().clone() for k, v in self.state_dict().items()
                }
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    if verbose:
                        print(f"Early stopping at epoch {epoch}, best val NLL: {best_val:.3f}")
                    break
        
        # Restore best
        if best_state is not None:
            self.load_state_dict(best_state)

        return {"best_val_nll":best_val}

    @torch.no_grad()
    def covariance(self, returns_df: pd.DataFrame) -> np.ndarray:
        """
        Compute fitted covariance matrix for the full series
        Args:
            returns_df: demeaned returns
        Output:
            array of shape (n_periods, n_assets, n_assets)
        """
        returns_np = returns_df.to_numpy(dtype=float)
        returns_tensor = to_tensor(returns_np, self.config.device)
        cov_matrix, _ = self.forward_sequence(returns_tensor, init_cov_matrix=self.H0_)
        return cov_matrix.cpu().numpy()
    
    def get_params(self) -> Dict[str, torch.Tensor]:
        a, b = self.ab()
        return {
            "C_static":self.C().detach().cpu(),
            "a": a.detach().cpu(),
            "b": b.detach().cpu(),
            "beta_swish":self.C_dynamic.beta.detach().cpu()
        }

    @torch.no_grad()
    def forecast_one_step(
        self,
        last_returns: np.ndarray,
        last_cov: np.ndarray
    ) -> np.ndarray:
        '''
        One-step ahead forecast with r_T and H_T
        Args: 
            last_returns: shape (n_assets, )
            last_cov: shape (n_assets, n_assets) (H_T)
        '''
        device = self.config.device
        returns = to_tensor(last_returns.reshape(1, -1), device)
        C = self.C()
        a, b = self.ab()
        C_dynamic = self.C_dynamic(returns)[0]

        cov_matrix_forecast = C @ C.T + C_dynamic @ C_dynamic.T + a * (returns @ returns.T) + b * to_tensor(last_cov, device)
        cov_matrix_forecast = cov_matrix_forecast * self.config.jitter * torch.eye(self.n_assets, dtype=torch.float64, device=device)

        return cov_matrix_forecast.cpu().numpy()
    
    @torch.no_grad()
    def forecast_multi_step(
        self,
        last_returns: np.ndarray,
        last_cov: np.ndarray,
        steps: int = 20,
        method: str = "zero"
    ) -> np.ndarray:
        """
        Args:
            method:
            - "zero": feed zeros for future returns
            - "simulate": simulate paths of future returns
        """
        device = self.config.device
        n_assets = self.n_assets
        forecasts = []

        r_curr = to_tensor(last_returns.reshape(1, -1), device)
        H_curr = to_tensor(last_cov, device)

        for step in range(steps):
            C = self.C()
            a, b = self.ab()
            
            # Dynamic C
            C_t = self.C_dynamic(r_curr)[0]

            if method == "zero" and step > 0:
                arch = torch.zeros(
                    n_assets, n_assets, dtype=torch.float64, device=device
                )
            else:
                arch = a * (r_curr.T @ r_curr)

            H_next = C @ C.T + C_t @ C_t.T + arch + b * H_curr
            H_next = H_next + self.config.jitter * torch.eye(n_assets, dtype=torch.float64, device=device)

            forecasts.append(H_next.cpu().numpy())

            # Update for next iteration
            H_curr = H_next
            if method == "zero":
                r_curr = torch.zeros_like(r_curr) # feed zeros
            elif method == "simulate":
                # sample one return path
                L = torch.linalg.cholesky(H_next)
                z = torch.randn(n_assets, 1, dtype=torch.float64, device=device)
                r_curr (L @ z).T 
        
        return np.stack(forecasts, axis=0)
    
def fit_lstm_bekk(
        returns_df: pd.DataFrame,
        hidden_size: Optional[int] = None,
        num_layers: int = 1,
        dropout: float = 0.1,
        lr: float = 0.001,
        epochs: int = 500,
        device: str = "cpu"
) -> LSTM_BEKK:
    n_assets = returns_df.shape[1]
    config = LSTM_BEKK_config(
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=dropout,
        lr=lr,
        epochs=epochs,
        device=device
    )
    model = LSTM_BEKK(n_assets=n_assets, config=config)
    model.fit(returns_df, verbose=True)

    return model

In [30]:
returns_df = train_df - train_df.mean() # Demean return

In [54]:
model = fit_lstm_bekk(
    returns_df=returns_df,
    hidden_size=3, # Same as number of assets
    num_layers=1,
    dropout=0.1,
    lr=0.001,
    epochs=500
)

[0000] train NLL : -552.885 | val NLL : -90.523
[0010] train NLL : -1667.438 | val NLL : -192.840
[0020] train NLL : -2128.596 | val NLL : -241.876
[0030] train NLL : -2459.089 | val NLL : -277.733
[0040] train NLL : -2731.069 | val NLL : -307.489
[0050] train NLL : -2969.607 | val NLL : -333.700
[0060] train NLL : -3186.307 | val NLL : -357.572
[0070] train NLL : -3387.524 | val NLL : -379.775
[0080] train NLL : -3577.077 | val NLL : -400.713
[0090] train NLL : -3757.383 | val NLL : -420.645
[0100] train NLL : -3930.028 | val NLL : -439.739
[0110] train NLL : -4096.069 | val NLL : -458.110
[0120] train NLL : -4256.362 | val NLL : -475.850
[0130] train NLL : -4411.490 | val NLL : -493.022
[0140] train NLL : -4562.002 | val NLL : -509.686
[0150] train NLL : -4708.380 | val NLL : -525.897
[0160] train NLL : -4851.068 | val NLL : -541.702
[0170] train NLL : -4990.392 | val NLL : -557.132
[0180] train NLL : -5127.019 | val NLL : -572.281
[0190] train NLL : -5261.056 | val NLL : -587.139
[0

In [55]:
# Parameters
params = model.get_params()
print(f"""
a, b = {params["a"].item()}, {params['b'].item()}
""")


a, b = 0.4007113830192244, 0.2556415151059981



In [ ]:
cov_matrix = model.covariance(returns_df)

In [ ]:
# One step forecast
last_return = returns_df.iloc[-1].to_numpy()
last_cov_matrix = cov_matrix[-1]
cov_matrix_1_step = model.forecast_one_step(
    last_returns=last_return,
    last_cov=last_cov_matrix
)
cov_matrix_1_step

array([[2.46974056e-09, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.09104797e-09, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 3.49805265e-09]])

In [58]:
# 20 stepts forecast
cov_matrix_20_steps = model.forecast_multi_step(
    last_returns=last_return,
    last_cov=last_cov_matrix,
    steps=20,
    method="zero"
)
cov_matrix_20_steps

array([[[0.24674678, 0.18141047, 0.12731048],
        [0.18141047, 0.40841957, 0.09459668],
        [0.12731048, 0.09459668, 0.34934679]],

       [[0.2465384 , 0.18285989, 0.12751626],
        [0.18285989, 0.41829586, 0.09473948],
        [0.12751626, 0.09473948, 0.35962741]],

       [[0.24648513, 0.18323042, 0.12756887],
        [0.18323042, 0.42082065, 0.09477599],
        [0.12756887, 0.09477599, 0.36225557]],

       [[0.24647151, 0.18332514, 0.12758231],
        [0.18332514, 0.42146609, 0.09478532],
        [0.12758231, 0.09478532, 0.36292743]],

       [[0.24646803, 0.18334936, 0.12758575],
        [0.18334936, 0.42163109, 0.09478771],
        [0.12758575, 0.09478771, 0.36309919]],

       [[0.24646714, 0.18335555, 0.12758663],
        [0.18335555, 0.42167327, 0.09478832],
        [0.12758663, 0.09478832, 0.3631431 ]],

       [[0.24646691, 0.18335713, 0.12758686],
        [0.18335713, 0.42168406, 0.09478847],
        [0.12758686, 0.09478847, 0.36315432]],

       [[0.24646686,

In [ ]:
# import pickle

# with open("lstm_bekk_cov_matrix.pkl", "wb") as f:
#     pickle.dump(cov_matrix, f)

# # Save model weights using torch.save
# torch.save(model.state_dict(), "lstm_bekk_model_state_dict.pt")

# Load model weights from file
model.load_state_dict(torch.load("lstm_bekk_model_state_dict.pt"))
model.eval()

In [ ]:
# Evaluate MVP

horizon = 20

last_return = returns_df.iloc[-1].to_numpy()
last_cov_matrix = cov_matrix[-1]

train_mean = train_df.mean()
demean_test_df = test_df - train_mean # Demean with in-sample mean
dates_test = demean_test_df.index

port_returns, port_vars, act_covs = [], [], [] # Containers

# Loop over test period in 20-days non-overlapping horizons
for start in range(0, len(demean_test_df) - horizon + 1, horizon):
    train_data = pd.concat([train_df, demean_test_df.iloc[:start]])
    
    # Forecast x step ahead
    cov_list = model.forecast_multi_step(
        last_returns=last_return,
        last_cov=last_cov_matrix,
        steps=horizon,
        method="zero"
    )

    # Aggregate to 20-days covariance forecast
    agg_covariance = sum(cov_list)

    # Get MVP weights
    mvp_weights, weights_dict = functions.minimum_variance_portfolio(agg_covariance , train_df)

    # Realized returns from next 20-days
    horizon_return = test_df[start:start+horizon]


    # Cummulative return
    port_return = np.array(horizon_return) @ mvp_weights
    port_returns.append(port_return.sum())

    # Actual covariance
    act_covariance =  horizon_return.T @ horizon_return
    act_var = mvp_weights.T @ act_covariance @ mvp_weights
    port_vars.append(act_var)
    act_covs.append(act_covariance)

    # Adjust for next iteration
    last_return = np.array(horizon_return.iloc[[-1]])[0]
    last_cov_matrix = np.array(act_covs[-1])

results =  pd.DataFrame({
    "date":dates_test[horizon-1::horizon][:len(port_returns)], # End of each horizon
    "realized_return":port_returns,
    "realized_variance":port_vars 
})

print(f"""
- Sharpe Ratio = {results["realized_return"].mean()/results["realized_return"].std()} \n
- Frobenius norm = {np.linalg.norm(agg_covariance - act_covariance, "fro")**2}
""")